# FPL Data Generator

This notebook scrapes played matches, then generates the required CSV files for the accompanying Tableau dashboards.

In [1]:
#Standard data manipulations
import pandas as pd
import numpy as np

from datetime import date
import sqlite3
import pickle

#Import custom functions
from PythonFunctions.apicallers import *
from PythonFunctions.sqlfunctions import *
from PythonFunctions.matchscrapers import *

#Suppress warnings from showing
import warnings
warnings.filterwarnings('ignore')

#Connect to the SQL database
conn = sqlite3.connect('Data/20_21fpl.db')
#Instantiate a cursor
c = conn.cursor()

Work out which matches we ought to have data for, which aren't present in the database.

In [2]:
to_run = suggested_match_ids()
print(to_run)

[58938, 58939, 58945]


Update data for these matches.

In [3]:
CoreDataUpdater(to_run, c, conn)

Stored match 0.0 (ID: 58938)
Match 58938 object successfully instantiated
Match 58938 shot detail dataframe successfully created
Error on row 0 - you may be trying to upload a duplicate row
Error on row 1 - you may be trying to upload a duplicate row
Error on row 2 - you may be trying to upload a duplicate row
Error on row 3 - you may be trying to upload a duplicate row
Error on row 4 - you may be trying to upload a duplicate row
Error on row 5 - you may be trying to upload a duplicate row
Error on row 6 - you may be trying to upload a duplicate row
Error on row 7 - you may be trying to upload a duplicate row
Error on row 8 - you may be trying to upload a duplicate row
Error on row 9 - you may be trying to upload a duplicate row
Error on row 10 - you may be trying to upload a duplicate row
Error on row 11 - you may be trying to upload a duplicate row
Error on row 12 - you may be trying to upload a duplicate row
Error on row 13 - you may be trying to upload a duplicate row
Error on row 

Error on row 26 - you may be trying to upload a duplicate row
Error on row 27 - you may be trying to upload a duplicate row
Error on row 28 - you may be trying to upload a duplicate row
Error on row 29 - you may be trying to upload a duplicate row
Error on row 30 - you may be trying to upload a duplicate row
Error on row 31 - you may be trying to upload a duplicate row
Error on row 32 - you may be trying to upload a duplicate row
Error on row 33 - you may be trying to upload a duplicate row
Error on row 34 - you may be trying to upload a duplicate row
Error on row 35 - you may be trying to upload a duplicate row
Error on row 36 - you may be trying to upload a duplicate row
Error on row 37 - you may be trying to upload a duplicate row
Error on row 38 - you may be trying to upload a duplicate row
Error on row 39 - you may be trying to upload a duplicate row
Error on row 40 - you may be trying to upload a duplicate row
Error on row 41 - you may be trying to upload a duplicate row
Error on

Sweep up matches that the updater might have missed.

In [4]:
MatchSweeper(c, conn)

Attempting to scrape [58900, 58938, 58939]

FAILURE: Match 58900 failed to scrape - may not have been played yet

Stored match 0.0 (ID: 58938)
Match 58938 object successfully instantiated
Match 58938 shot detail dataframe successfully created
Error on row 0 - you may be trying to upload a duplicate row
Error on row 1 - you may be trying to upload a duplicate row
Error on row 2 - you may be trying to upload a duplicate row
Error on row 3 - you may be trying to upload a duplicate row
Error on row 4 - you may be trying to upload a duplicate row
Error on row 5 - you may be trying to upload a duplicate row
Error on row 6 - you may be trying to upload a duplicate row
Error on row 7 - you may be trying to upload a duplicate row
Error on row 8 - you may be trying to upload a duplicate row
Error on row 9 - you may be trying to upload a duplicate row
Error on row 10 - you may be trying to upload a duplicate row
Error on row 11 - you may be trying to upload a duplicate row
Error on row 12 - you m

Error on row 22 - you may be trying to upload a duplicate row
Error on row 23 - you may be trying to upload a duplicate row
Error on row 24 - you may be trying to upload a duplicate row
Error on row 25 - you may be trying to upload a duplicate row
Error on row 26 - you may be trying to upload a duplicate row
Error on row 27 - you may be trying to upload a duplicate row
Error on row 28 - you may be trying to upload a duplicate row
Error on row 29 - you may be trying to upload a duplicate row
Error on row 30 - you may be trying to upload a duplicate row
Error on row 31 - you may be trying to upload a duplicate row
Error on row 32 - you may be trying to upload a duplicate row
Error on row 33 - you may be trying to upload a duplicate row
Error on row 34 - you may be trying to upload a duplicate row
Error on row 35 - you may be trying to upload a duplicate row
Error on row 36 - you may be trying to upload a duplicate row
Error on row 37 - you may be trying to upload a duplicate row
Error on

# CSV Creation

In [5]:
#Bring in XG functions now (note this is brought in after the new goal data is known)
from PythonFunctions.xg import *

#Generate a player detail dataframe, with the XG stats 
df_pm = pd.merge(df_pm_generator(),
                 api_stat_generator(),
                 on=['MatchID','Player']).drop(['TableIndex','Minutes'], axis=1)

df_tm = df_tm_generator()

In [6]:
df_pm.tail(10)

,MatchID,Player,GameWeek,ForTeam,AgainstTeam,RelativeStrength,Goals,ShotsOnTarget,ShotsInBox,CloseShots,...,Position,Date,BPS,CleanSheet,MinutesPlayed,NetTransfersIn,Points,Price,Saves,SelectedBy
2452,58939,Pedro Neto,5,Wolverhampton Wanderers,Leeds United,1,0,0,1,0,...,MID,2020-10-19,5,1.0,88.0,14275,3,5.5,0.0,95820
2453,58939,Bruno Jordao,5,Wolverhampton Wanderers,Leeds United,1,0,0,0,0,...,MID,2020-10-19,0,0.0,0.0,-2657,0,4.4,0.0,23919
2454,58939,Leonardo Campana,5,Wolverhampton Wanderers,Leeds United,1,0,0,0,0,...,FWD,2020-10-19,0,0.0,0.0,-2331,0,4.9,0.0,23921
2455,58939,Fabio Silva,5,Wolverhampton Wanderers,Leeds United,1,0,0,0,0,...,FWD,2020-10-19,0,0.0,0.0,-10723,0,5.3,0.0,50687
2456,58939,Fernando Marcal,5,Wolverhampton Wanderers,Leeds United,1,0,0,0,0,...,DEF,2020-10-19,2,0.0,1.0,-1646,1,4.9,0.0,13471
2457,58939,Vitor Vitinha,5,Wolverhampton Wanderers,Leeds United,1,0,0,0,0,...,MID,2020-10-19,0,0.0,0.0,-314,0,4.9,0.0,2510
2458,58939,Ki-Jana Hoever,5,Wolverhampton Wanderers,Leeds United,1,0,0,0,0,...,DEF,2020-10-19,0,0.0,0.0,171,0,4.5,0.0,2443
2459,58939,Nelson Semedo,5,Wolverhampton Wanderers,Leeds United,1,0,0,0,0,...,DEF,2020-10-19,24,1.0,90.0,36935,6,5.5,0.0,125665
2460,58939,Meritan Shabani,5,Wolverhampton Wanderers,Leeds United,1,0,0,0,0,...,MID,2020-10-19,0,0.0,0.0,249,0,4.5,0.0,2163
2461,58939,Rayan Ait Nouri,5,Wolverhampton Wanderers,Leeds United,1,0,0,0,0,...,DEF,2020-10-19,0,0.0,0.0,714,0,5.0,0.0,1033


In [7]:
df_tm.tail(10)

,MatchID,ForTeam,AgainstTeam,RelativeStrength,GameWeek,Home,Possession,Goals,ShotsOnTarget,TotalShots,...,ShotsLeft,ShotsRight,ShotsInBoxConceded,CloseShotsConceded,HeadersConceded,ShotsCentreConceded,ShotsLeftConceded,ShotsRightConceded,XG,XGC
80,58941,Manchester City,Arsenal,1,5,Home,58.5,1,5,13,...,0,4,5,5,2,3,2,0,2.227552,1.134218
81,58941,Arsenal,Manchester City,-1,5,Away,41.5,0,3,11,...,2,0,8,6,1,4,0,4,1.134218,2.227552
82,58942,Newcastle United,Manchester United,-1,5,Home,36.4,1,4,7,...,1,0,13,14,2,5,7,2,1.573484,3.935006
83,58942,Manchester United,Newcastle United,1,5,Away,63.6,4,14,28,...,7,2,2,4,0,1,1,0,3.935006,1.573484
84,58943,Sheffield United,Fulham,1,5,Home,41.5,1,6,10,...,1,1,8,8,5,7,1,0,1.950705,2.680106
85,58943,Fulham,Sheffield United,-1,5,Away,58.5,1,6,15,...,1,0,5,6,3,3,1,1,2.680106,1.950705
86,58944,Tottenham Hotspur,West Ham United,1,5,Home,51.2,3,6,11,...,0,1,8,7,3,3,5,0,2.513048,1.961972
87,58944,West Ham United,Tottenham Hotspur,-1,5,Away,48.8,3,4,13,...,5,0,3,8,1,2,0,1,1.961972,2.513048
88,58937,Crystal Palace,Brighton and Hove Albion,1,5,Home,34.3,1,1,1,...,0,0,11,5,4,8,2,1,0.789474,2.388551
89,58937,Brighton and Hove Albion,Crystal Palace,-1,5,Away,65.7,1,3,20,...,2,1,0,1,0,0,0,0,2.388551,0.789474


In [8]:
df_matches.drop('TableIndex', axis=1).to_csv('Outputs/fixtures.csv')
df_pm.to_csv('Outputs/player_data.csv')
df_tm.to_csv('Outputs/team_data.csv')

In [9]:
def next_x_games(team, x, df_tm = df_tm):
    
    '''
    returns a dataframe for the next x games for a given team
    '''

    df_temp = df_matches[(df_matches['Team']==team)
              &~(df_matches['MatchID']).isin(list(df_tm['MatchID']))].head(x)
    
    return df_temp[['MatchID','Opposition']]



def prev_x_games(team, x, df_tm = df_tm):
    
    #Create a dataframe with the right team
    return df_tm.loc[df_tm['ForTeam']==team].sort_values('GameWeek', ascending=False).head(x)



def weighted_prev_xg(team, x, final_weight = 0.5, df_tm = df_tm):
    
    df_temp = prev_x_games(team=team, x=x, df_tm=df_tm)
    
    #Calculate weights that we will apply to historical values to get an
    #average that emphasises recent performance
    x = len(df_temp)
    
    df_temp['Weights'] = [1-i*(1-final_weight)/(x-1) for i in range(x)]
    df_temp['Weights'] = df_temp['Weights'].map(lambda x: x/df_temp['Weights'].sum())
    
    df_temp['XG_Weighted'] = df_temp['Weights'] * df_temp['XG']
    df_temp['XGC_Weighted'] = df_temp['Weights'] * df_temp['XGC']
    
    xg = df_temp['XG_Weighted'].sum()
    xgc = df_temp['XGC_Weighted'].sum()

    return {'xg_weighted':xg,
           'xgc_weighted':xgc}


def upcoming_xg_index(team, x_back, x_forward,
                      final_weight = 0.5, forward_weight = 0.25,
                      df_tm = df_tm):
    '''
    Creates a dataframe for a team, which has the XG and XGC of itself and
    upcoming teams, as calculated by the weighted functions
    '''
    
    df_stub = next_x_games(team, x=x_forward, df_tm = df_tm)
    
    team_stats = weighted_prev_xg(team, x_back, final_weight = final_weight)
    df_stub['xg'] = [team_stats['xg_weighted'] for i in range(x_forward)]
    df_stub['xgc'] = [team_stats['xgc_weighted'] for i in range(x_forward)]
    
    df_stub['Opp_xg'] = [weighted_prev_xg(i, x_back,
                                          final_weight = final_weight)['xg_weighted']
                          for i in df_stub['Opposition']]
    
    df_stub['Opp_xgc'] = [weighted_prev_xg(i, x_back,
                                           final_weight = final_weight)['xgc_weighted']
                          for i in df_stub['Opposition']]
        
    df_stub['xg_index'] = df_stub['xg'] * df_stub['Opp_xgc']
    df_stub['xgc_index'] = df_stub['xgc'] * df_stub['Opp_xg']
    
    x = len(df_stub)
    
    df_stub['Weights'] = [1-i*(1-forward_weight)/(x-1) for i in range(x)]
    df_stub['Weights'] = df_stub['Weights'].map(lambda x: x/df_stub['Weights'].sum())
    
    xg_index_avg = (df_stub['Weights'] * df_stub['xg_index']).sum()
    xgc_index_avg = (df_stub['Weights'] * df_stub['xgc_index']).sum()
    
    return xg_index_avg, xgc_index_avg


def upcoming_xg_index_df(x_back = 5, x_forward = 6,
                      final_weight = 0.6, forward_weight = 0.4,
                      df_tm = df_tm):
    
    df_temp = pd.DataFrame(columns = ['Team','XGIndex','XGCIndex'])
    
    for i in df_teams['CommentName']:
        indices = upcoming_xg_index(i, x_back, x_forward,
                                    final_weight, forward_weight, df_tm)
        
        new_row = pd.DataFrame({'Team':[i],
                                'XGIndex':[indices[0]],
                                'XGCIndex':[indices[1]]})
        
        df_temp = pd.concat([df_temp, new_row]).reset_index(drop=True)
        
    return df_temp

In [10]:
df_xg_index = upcoming_xg_index_df()
df_xg_index.to_csv('Outputs/xg_index.csv')

In [11]:
df_xg_index.sort_values('XGIndex', ascending=False)

,Team,XGIndex,XGCIndex
16,Tottenham Hotspur,7.530345,3.355634
4,Chelsea,6.860298,3.323874
6,Everton,6.577486,3.295235
10,Liverpool,5.350965,4.686632
2,Brighton and Hove Albion,4.954547,3.318905
18,West Ham United,4.954038,3.240913
15,Southampton,4.783479,3.475824
1,Aston Villa,4.691987,2.813854
8,Leicester City,4.394652,3.381923
12,Manchester United,4.353994,6.099922


In [12]:
df_xg_index.sort_values('XGCIndex')

,Team,XGIndex,XGCIndex
1,Aston Villa,4.691987,2.813854
18,West Ham United,4.954038,3.240913
6,Everton,6.577486,3.295235
2,Brighton and Hove Albion,4.954547,3.318905
4,Chelsea,6.860298,3.323874
16,Tottenham Hotspur,7.530345,3.355634
8,Leicester City,4.394652,3.381923
15,Southampton,4.783479,3.475824
7,Fulham,3.469553,3.803805
19,Wolverhampton Wanderers,3.481759,3.848551


Ta dah.